<a href="https://colab.research.google.com/github/moizca/Research_Paper_Implementation/blob/main/Qlearning_implementation_using_with_openai_gym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install gym

In [2]:
pip install tensorflow==1.15.0

     |████████████████████████████████| 412.3 MB 16 kB/s 
     |████████████████████████████████| 50 kB 5.7 MB/s 
     |████████████████████████████████| 503 kB 76.4 MB/s 
     |████████████████████████████████| 3.8 MB 45.8 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=7b92f7e5597d400dad7e0a9f56df324cfcb7235ce97a9d23bcece3a67ad67cce
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succes

In [3]:
!sudo apt-get update
!sudo apt-get install cmake libopenmpi-dev python3-dev zlib1g-dev

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,812 kB]
Ign:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:13 https://cloud.r-project.org/bin/linu

In [4]:
pip install stable-baselines[mpi]

     |████████████████████████████████| 240 kB 16.6 MB/s 
     |████████████████████████████████| 2.5 MB 45.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.2-cp37-cp37m-linux_x86_64.whl size=2183198 sha256=73a6caa0b947fe08c0f614e43fd1cd198cba29242ebd8067538e9e27bf6046ec
  Stored in directory: /root/.cache/pip/wheels/62/d6/0a/91ea5f998589bc72ca3fcf425c0e364baf8536753e1cd26ea4
Successfully built mpi4py


In [1]:
import numpy as np

In [2]:
num_episodes = 2000
max_steps_per_episode = 6

learning_rate = np.float64(0.8)
discount_rate = np.float64(0.6)

demand1=[[4,4,4],[3,3,3],[2,2,2],[1,1,1],[6,6,6],[7,7,7]]
demand=np.array(demand1)
ddd=demand1[0][0]+demand1[1][0]+demand1[2][0]+demand1[3][0]+demand1[4][0]+demand1[5][0]+1
ddd_list=list(range(ddd))
from itertools import product
perm = list(product(ddd_list, repeat = 3))
ddd_list_actions=[0,1]
perm_a=list(product(ddd_list_actions, repeat = 3))
perm_actions=[]
for ptp in perm_a:
  if sum(ptp)==1 or sum(ptp)==0:
    perm_actions.append(ptp)
perm.sort()
perm_actions.sort()

In [3]:
import basic_env
env = basic_env.BasicEnv()

In [4]:
from stable_baselines.common.env_checker import check_env
check_env(env)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



/usr/local/lib/python3.7/dist-packages/stable_baselines/__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation."


In [5]:
action_space_size = env.action_space.n
q_table = np.zeros((len(perm), action_space_size))
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 ...
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [9]:
import random
from math import exp
# Q-Learning algorithm
rewards_all_episodes=[]
for episode in range(num_episodes):
#   exploration_rate=0.5/(1+exp(10*((episode+1)-0.4*num_episodes))/num_episodes)
    exploration_rate=episode/num_episodes
    done = False
    rewards_current_episode = 0
    sstep=0
    stat=env.reset()
    for step in range(max_steps_per_episode):
        # Exploration -exploitation trade-off
        stat=int(stat)
        state = perm[stat]
        state=np.array(state).reshape(3,1).astype(np.int)
        if random.uniform(0,1) > exploration_rate:
            act = np.argmax(q_table[stat,:])
        else:
            act = env.action_space.sample()

        new_stat, reward, done, info = env.step(act)
        action = perm_actions[act]
        # Update Q-table for Q(s,a)
        q_table[stat, act] = (1 - learning_rate) * q_table[stat, act] + \
            learning_rate * (reward + discount_rate * np.max(q_table[new_stat,:]))
            
        stat = new_stat
        rewards_current_episode += reward
        
        if done == True: 
            break
    
    rewards_all_episodes.append(rewards_current_episode)
# Calculate and print the average reward per 10 episodes
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes), num_episodes / 100)
count = 100
print("********** Average  reward per thousand episodes **********\n")

for r in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(r / 100)))
    count += 100
    
# Print updated Q-table
print("\n\n********** Q-table **********\n")
print(q_table)

********** Average  reward per thousand episodes **********

100 :  -5.539211986890378e+30
200 :  -3.445755524528738e+30
300 :  -3.0797211522880134e+30
400 :  -2.317910707781936e+30
500 :  -3.9625308486557183e+30
600 :  -1.1938803093147599e+30
700 :  -1.1285014204842522e+30
800 :  -1.1886618006989584e+30
900 :  -1.4144669694633126e+30
1000 :  -1.268479875113574e+30
1100 :  -2.6107460061556014e+30
1200 :  -2.4560322558591022e+30
1300 :  -2.087456923963561e+30
1400 :  -2.3316797642881508e+30
1500 :  -2.3401116510804267e+30
1600 :  -3.282796461494883e+30
1700 :  -3.169016199567589e+30
1800 :  -4.1230507281124173e+30
1900 :  -3.969751743325161e+30
2000 :  -4.445631231089973e+30


********** Q-table **********

[[-2.37338852e+27 -3.94680536e+27 -2.51425818e+27 -2.37346037e+27]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.000000

In [17]:
print(q_table)
q_table[77,:]
q_table_values=[]
for q in q_table:
  q =  [abs(ele) for ele in q]
  if sum(q)>0:
    q_table_values.append(q)
print(q_table_values)
print(len(q_table_values))

[[-2.37338852e+27 -3.94680536e+27 -2.51425818e+27 -2.37346037e+27]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]]
[[2.3733885236019992e+27, 3.9468053558279634e+27, 2.514258179496727e+27, 2.3734603661949617e+27], [3.955232842225669e+27, 9.278876028388982e+27, 4.191661552428067e+27, 9.278876028381593e+27], [2.2463244226866905e+28, 9.278876028388982e+27, 9.278876028383124e+27, 4.1904067410465345e+27], [1.6718699212585303e+28, 9.278876028388982e+27, 5.902792150877679e+27, 9.278876028388982e+27], [4.8389776690578124e+27, 1.6737290596184612e+28, 2.4610162604404423e+28, 3.9556473476710927e+27], [1.0214117742086857e+29, 1.5464793380645841e+28, 1.5464793380648304e+28, 9.69987112388988e+28], [4.064753103942